### Import Libraries

In [1]:
from bs4 import BeautifulSoup as bs
import requests

### Task 1: Get the wiki page for Marvel Cinematic Universe

In [2]:
wiki_address = "https://en.wikipedia.org"
marvel_address = wiki_address + "/wiki/List_of_Marvel_Cinematic_Universe_films"
r = requests.get(marvel_address)

# Convert to beautiful soup object
soup = bs(r.content)

### Task 2: Select relevant data from the page

In [14]:
# Get tables from the page
movie_table = soup.find_all(class_="wikitable plainrowheaders")[:2]
movie_rows = []
for index, table in enumerate(movie_table):
    if index == 0:
        movie_rows.append(table.find_all("tr"))    
    movie_rows.append(table.find_all("tr")[1:])

def get_subsect(main_row):
    sub_rows = main_row.find_all("li")
    subsect_list = []
    for row in sub_rows:
        row_text = row.find(class_="toctext").get_text()
        if row.find("ul"):
            subsect_list.append(row_text + ":")
            subsect_list.append(get_subsect(row))
        else:
            subsect_list.append(row_text)
    return subsect_list

def get_linked_content(address):
    r_c = requests.get(address)
    soup_c = bs(r_c.content)
    
    toc = soup_c.find(class_="toc").find("ul")
    toc_rows = toc.find_all("li")

    content_list = []
    for row in toc_rows:
        row_text = row.find(class_="toctext").get_text()
        if row_text == 'See also':
            break
        if row.find("ul"):
            content_list.append(row_text + ":")
            content_list.append(get_subsect(row))
            continue

        content_list.append(row_text)

    return content_list

def get_box_info(row):
    row_text = row.find("td").get_text()
    if "Full list" in row_text:
        content_address = wiki_address + row.find(href = True)['href']
        return get_linked_content(content_address)
    elif row.find("li"):
        return [li.get_text() for li in row.find_all("li")]
    else:
        return row_text

def get_movie_page_info(link):
    r_c = requests.get(wiki_address + link)
    soup_c = bs(r_c.content)
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")

    movie_info = {}

    for i, row in enumerate(info_rows):
        if i == 0:
            movie_info["Title"] = row.find("th").get_text()
        elif i == 1:
            continue
        else:
            try:
                movie_key = row.find("th").get_text()
                movie_value = get_box_info(row)
                movie_info[movie_key] = movie_value
            except:
                continue
    return movie_info

# Collect table rows and row links
table_rows = []
box_rows = []
for table in movie_rows:
    for row in table:
        try:
            if(row.find("th")['scope'] == "row") and row not in table_rows:
                table_rows.append(row)

                # Data from individual movie pages are collected to be merged later.
                link = row.find(href = True)['href']
                box_row = get_movie_page_info(link)
                box_rows.append(box_row)
        except:
            continue

### Task 3: Organize the data

In [20]:
# Context units are collected from the header of the table.
key_list = []
for col in movie_rows[0][0].find_all("th"):
    key_list.append(col.get_text().replace("\n", ""))

# Data is collected from columns and paired with context units 
# in a movie class, which is then stored in a list.
movie_list = []
for i, row in enumerate(table_rows):
    value_list = []
    for col in row.find_all("th") + row.find_all("td"):
        col_text = col.get_text()

        # Clean the data
        if "[" in col_text:
            col_text = col_text.split("[")[0] + "\n"
            
        value_list.append(col_text.replace("\n", ""))
    movie_list.append({**dict(zip(key_list, value_list)), **box_rows[i]})

### Task 4: Cleanup data

In [23]:
movie_list[0]

{'Film': 'Iron Man',
 'U.S. release date': 'May 2, 2008',
 'Director(s)': 'Jon Favreau',
 'Screenwriter(s)': 'Mark Fergus & Hawk Ostby and Art Marcum & Matt Holloway',
 'Producer(s)': 'Avi Arad and Kevin Feige',
 'Title': 'Marvel Cinematic Universe films',
 'Based on': 'Characters publishedby Marvel Comics',
 'Produced by': ['Kevin Feige[a]', 'Gale Anne Hurd (TIH)', 'Amy Pascal (SM)'],
 'Starring': 'See below',
 'Productioncompanies': ['Marvel Studios',
  'Columbia Pictures(SM; 2017–present)'],
 'Distributed by': ['Paramount Pictures(2008–2011)',
  'Universal Pictures(TIH; 2008)',
  'Walt Disney StudiosMotion Pictures(2012–present)',
  'Sony Pictures Releasing (SM; 2017–present)'],
 'Release date': '2008–present',
 'Country': 'United States',
 'Language': 'English',
 'Budget': 'Total (27 films):$5.100,5–5.558\xa0billion',
 'Box office': 'Total (27 films):$25.694\xa0billion'}

### Task 5: Print and save the results

In [22]:
# Print results
def print_dict(dict):
        print("\n")
        for key in dict:
            print(key + ": " + dict[key] + "\n")
        print("\n")

# for movie in movie_list:
#     print_dict(movie)

# Function to save the list as text file
def save_txt(filename, list):
    with open(filename + ".txt", 'w') as f: 
        for dict in list:
            f.write('\n')
            for key in dict: 
                f.write('%s: %s' % (key, dict[key]))

# Function to save the list as csv file
def save_csv(filename, data_list):
    import pandas as pd

    df = pd.DataFrame(data_list)

    df.to_csv(filename + ".csv")

# Function to save the list as excel file
def save_xlsx(filename, data_list):
    import pandas as pd

    df = pd.DataFrame(data_list)

    writer = pd.ExcelWriter(filename + ".xlsx", engine='xlsxwriter')
    df.to_excel(writer,sheet_name = filename, index=False)
    writer.save() 

# Save the results
save_txt("MCU", movie_list)
save_csv("MCU", movie_list)
save_xlsx("MCU", movie_list)